In [1]:
import torch
import pandas as pd
import torch.optim as optim
from torch.nn import Parameter
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.w1 = Parameter(torch.ones(24) * 0.98) 
        self.w2 = Parameter(torch.ones(24) * 1.2)

    def forward(self, oracle_demand, oracle_solar, oracle_price):
        bid_demand = (oracle_demand - oracle_solar) * self.w1
        bid_price = oracle_price * self.w2
        bid_price.data.clamp_(max=7)
        return bid_demand, bid_price

In [3]:
demand_train_oracle = pd.read_csv('./Data/Demand_Train_pred.csv', header=None)
demand_train_actual = pd.read_csv('./Data/Demand_Train.csv', header=None)
solar_train_oracle = pd.read_csv('./Data/Solar_Train_pred.csv', header=None)
solar_train_actual = pd.read_csv('./Data/Solar_Train.csv', header=None)
price_train_oracle = pd.read_csv('./Data/Price_Train_pred.csv', header=None)
price_train_actual = pd.read_csv('./Data/Price_Train.csv', header=None)

In [4]:
def cost(bid_quantities, bid_prices, demands, solar_outputs, prices):
    BATTERY_CAPACITY = 25
    BATTERY_CAPACITY_PER_HOUR = 5
    BATTERY_EFFICIENCY = 0.8
    DISCOM_RATE = 7
    battery = Variable(torch.Tensor([0]))
    cost = Variable(torch.Tensor([0]))
    result = []
    for bid_quantity, bid_price, demand, solar_output, price in zip(bid_quantities.contiguous().view(-1), bid_prices.contiguous().view(-1), demands.contiguous().view(-1), solar_outputs.contiguous().view(-1), prices.contiguous().view(-1)):
        energy_from_sun = torch.min(solar_output, demand)
        bid_won = bid_price > price
        energy_from_market = bid_won.float() * bid_quantity
        energy_from_battery = torch.min(torch.clamp((demand - energy_from_sun - energy_from_market) / BATTERY_EFFICIENCY, min=0),
                                        torch.clamp(battery, max=BATTERY_CAPACITY_PER_HOUR)) * BATTERY_EFFICIENCY
        energy_from_DISCOM = torch.clamp(demand - energy_from_sun - energy_from_market - energy_from_battery, min=0)
        
        residual_energy = torch.clamp(energy_from_market + energy_from_sun - demand, min=0)
        battery = battery + torch.clamp(residual_energy, max=BATTERY_CAPACITY_PER_HOUR) - energy_from_battery / BATTERY_EFFICIENCY
        battery = torch.clamp(battery, min=0, max=BATTERY_CAPACITY)
        
        cost += energy_from_market * bid_price + energy_from_DISCOM * DISCOM_RATE
    return cost

In [5]:
net = Net()

In [6]:
data_size = demand_train_actual.values.shape[0]
epochs = 1
batch_size = 50
learning_rate = 0.00001

In [7]:
for _ in range(epochs):
    start = 0
    end = batch_size
    epoch_loss = 0
    while end <= data_size:
        net.zero_grad()
        bid_quantities, bid_prices = net(Variable(torch.Tensor(demand_train_oracle.values[start:end])), 
                              Variable(torch.Tensor(solar_train_oracle.values[start:end])), 
                              Variable(torch.Tensor(price_train_oracle.values[start:end])))
        loss = cost(bid_quantities, bid_prices,
             Variable(torch.Tensor(demand_train_actual.values[start:end])),
             Variable(torch.Tensor(solar_train_actual.values[start:end])),
             Variable(torch.Tensor(price_train_actual.values[start:end])))
        loss.backward()
        for f in net.parameters():
            f.data.sub_(f.grad.data * learning_rate)
            f.data.clamp_(min=0.5, max=2)
        epoch_loss += loss.data[0]
        
        start += batch_size
        end += batch_size
    print(epoch_loss * batch_size / data_size)

525818.9809027778


In [8]:
demand_pub_oracle = pd.read_csv('./Data/Demand_LB_pred.csv', header=None)
solar_pub_oracle = pd.read_csv('./Data/Solar_LB_pred.csv', header=None)
price_pub_oracle = pd.read_csv('./Data/Price_LB_pred.csv', header=None)

In [9]:
print(list(net.parameters()))

[Parameter containing:
 1.1564
 1.1946
 1.2155
 1.2301
 1.1949
 1.1656
 1.0361
 0.9131
 0.9393
 0.9719
 0.9978
 1.0002
 1.0410
 1.0243
 1.0171
 1.0128
 1.0413
 1.0260
 0.9991
 0.9985
 0.9780
 1.0474
 1.0799
 1.1447
[torch.FloatTensor of size 24]
, Parameter containing:
 0.8803
 0.8912
 0.9048
 0.9249
 0.9114
 0.8845
 0.8684
 0.8656
 0.8686
 0.8627
 0.8611
 0.8664
 0.8680
 0.8664
 0.8687
 0.8645
 0.8684
 0.8674
 0.8663
 0.8424
 0.8621
 0.8610
 0.8689
 0.8723
[torch.FloatTensor of size 24]
]


In [10]:
def submit(name):
    url = 'http://interiit.tech:3000/upload'
    files = {'file': (name, open(name, 'rb'), '.text/csv', {'Content-Disposition': 'form-data'})}
    headers = {'Host': 'interiit.tech:3000', 'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:58.0) Gecko/20100101 Firefox/58.0',\
               'Accept': 'application/json, text/plain, */*', 'Accept-Language': 'en-US,en;q=0.5', 'Accept-Encoding': 'gzip, deflate',\
               'Referer': 'http://interiit.tech:3000/', 'Origin':'http://interiit.tech:3000'}
    r = requests.post(url, files=files, headers=headers)
    
def score(team_id):
    return pd.read_csv('http://interiit.tech:3000/results.csv').loc[team_id]['Score']

In [11]:
bid_quantity = ((demand_pub_oracle - solar_pub_oracle).values * net.state_dict()['w1'].numpy()).flatten()
bid_price = (price_pub_oracle.values * net.state_dict()['w2'].numpy()).flatten()
submission = pd.DataFrame([bid_price, bid_quantity])
submission.T.to_csv('23.csv', header=None, index=None)
score(23)

320343